In [ ]:
# Standard Libraries
import numpy as np
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

# ABM Framework (Mesa 2.1.0)
from mesa import Agent, Model
from mesa.time import RandomActivation  # Correct import for Mesa 2.1.0
from mesa.datacollection import DataCollector

# --- GLOBAL MODEL PARAMETERS (Synthesized from Thesis Data) ---

# Contextual Parameters (from Appendix E & Chapter 3.2)
ANNUAL_SWM_BUDGET = 1_500_000 # ₱1,500,000
QUARTERLY_BUDGET = ANNUAL_SWM_BUDGET / 4 # ₱375,000
NUM_BARANGAYS = 7
NUM_EPISODES = 10000 # Number of training runs for the RL agent
STEPS_PER_EPISODE = 4 # 4 quarters per year (episode)

# Barangay Data (Households, Local_Budget, Initial_Compliance)
BARANGAY_DATA = {
    "LianganEast": (608, 30000, 0.65), 
    "Poblacion": (700, 40000, 0.10),
    "Esperanza": (550, 25000, 0.08),
    "Binuni": (400, 20000, 0.05),
    "Demologan": (450, 22000, 0.07),
    "Mati": (500, 30000, 0.12),
    "Babalaya": (350, 18000, 0.09),
}

# Cost Parameter Estimation
COST_PER_ENFORCER_QUARTER = 2000 
COST_PER_IEC_INTENSITY_UNIT = 5000 
PAYOUT_PER_COMPLIANT_HOUSEHOLD = 50 

# RL Reward Weights (Chapter 3.4.3)
ALPHA_COMPLIANCE = 100.0   # WAS 10.0 (Boost reward for high compliance)
BETA_COST = 0.000001       # WAS 0.00001 (Make spending 10x cheaper penalty-wise)
GAMMA_DEFICIT = 50.0       # Keep high to prevent bankruptcy

# TPB Parameter Defaults (Chapter 3.2.1)
TPB_WEIGHTS = {
    'wA': 0.45,  # Attitude
    'wSN': 0.35, # Subjective Norms 
    'wPBC': 0.20 # Perceived Behavioral Control
}

# Psychological Reactance Threshold (Chapter 3.2.1)
REACTANCE_THRESHOLD = 0.25

In [2]:
# --- CELL 2: HOUSEHOLD AGENT (BALANCED) ---
class HouseholdAgent(Agent):
    def __init__(self, unique_id, model, initial_income, initial_edu_level, barangay_id):
        super().__init__(unique_id, model)
        self.barangay_id = barangay_id
        self.income = initial_income
        self.education = initial_edu_level
        self.is_compliant = 0
        
        # TPB Internal State
        self.attitude = random.uniform(0.3, 0.7)
        self.subj_norm = random.uniform(0.2, 0.6)
        self.perc_b_control = random.uniform(0.4, 0.8)

    def calculate_utility(self, fine_magnitude, incentive_payout, enforcement_risk):
        wA, wSN, wPBC = TPB_WEIGHTS['wA'], TPB_WEIGHTS['wSN'], TPB_WEIGHTS['wPBC']
        
        # 1. Psychological Utility
        # BOOST: Increased multiplier from 50 to 60. 
        # Avg person (0.5) now starts with 30 points (closer to the 50 target).
        Raw_Psych = (wA * self.attitude) + (wSN * self.subj_norm) + (wPBC * self.perc_b_control)
        Psych_Points = Raw_Psych * 60 
        
        # 2. Financial Sensitivity
        sensitivity = 1.0 - (self.income / self.model.MAX_INCOME_PROXY)
        
        # 3. Policy Utility
        
        # Incentive: REMOVED the 0.5 dampener.
        # Poor person (0.8 sens) gets 50 * 0.8 = 40 points. (Instant compliance if they get money)
        Util_Incentive = incentive_payout * sensitivity 
        
        # Fine: Added "Perceived Risk" Multiplier.
        # People tend to overestimate police presence. 
        # If Risk is 5% (0.05), we multiply by 4 to simulate "Fear Factor".
        # 500 * 0.05 * 4.0 * sensitivity = 100 * sensitivity points.
        Util_Fine_Avoidance = fine_magnitude * enforcement_risk * sensitivity * 4.0
        
        # Total Score
        Total_Utility = Psych_Points + Util_Incentive + Util_Fine_Avoidance
        
        return Total_Utility

    def update_tpb_constructs(self, policy_action, neighborhood_fined_rate, avg_compliance):
        self.attitude += 0.01 * (policy_action['IEC'] / 20000) * self.model.IEC_EFFECTIVENESS
        if neighborhood_fined_rate > REACTANCE_THRESHOLD:
            self.attitude -= 0.05 
        self.subj_norm = (0.9 * self.subj_norm) + (0.1 * avg_compliance)
        self.perc_b_control += 0.005 * ((policy_action['IEC'] + policy_action['ENFORCE'])/50000)

        self.attitude = np.clip(self.attitude, 0, 1)
        self.subj_norm = np.clip(self.subj_norm, 0, 1)
        self.perc_b_control = np.clip(self.perc_b_control, 0, 1)

    def step(self, fine_magnitude, incentive_payout, enforcement_risk):
        utility = self.calculate_utility(fine_magnitude, incentive_payout, enforcement_risk)
        
        # THRESHOLD: 50 Points
        self.is_compliant = 1 if utility > 50 else 0
        return self.is_compliant

In [3]:
class BarangayAgent(Agent):
    """
    Intermediate implementation layer. Receives LGU funds and manages local 
    implementation (enforcement, IEC, incentives).
    """
    def __init__(self, unique_id, model, name, households, local_budget, initial_compliance):
        super().__init__(unique_id, model)
        self.name = name
        self.households = households 
        self.local_budget = local_budget 
        self.current_compliance_rate = initial_compliance
        self.last_enforcement_fined_count = 0
        self.policy_allocation = {'IEC': 0, 'ENFORCE': 0, 'INCENTIVE': 0}
        
    def aggregate_and_report(self):
        """Aggregates household compliance and calculates local policy effects."""
        
        # 1. Calculate Aggregate Compliance
        compliant_count = sum(h.is_compliant for h in self.households)
        self.current_compliance_rate = compliant_count / len(self.households)
        
        # 2. Calculate Enforcement/Fine Effectiveness
        non_compliant_count = len(self.households) - compliant_count
        
        # Enforcement effectiveness is proportional to spending.
        enforcement_staff = self.policy_allocation['ENFORCE'] / COST_PER_ENFORCER_QUARTER
        
        # Probability of being caught/fined
        fined_prob = min(1.0, 0.05 * enforcement_staff) 
        
        # Number of households fined this quarter
        fined_count = sum(1 for _ in range(non_compliant_count) if random.random() < fined_prob)
        self.last_enforcement_fined_count = fined_count

        # Calculate incentive cost and successful enforcement cost
        incentive_cost = compliant_count * PAYOUT_PER_COMPLIANT_HOUSEHOLD
        enforcement_revenue = fined_count * self.model.LGU_FINE_MAGNITUDE # Revenue from fines
        
        return self.current_compliance_rate, self.last_enforcement_fined_count, incentive_cost, enforcement_revenue

In [4]:
# --- CELL 4: RL AGENT ---
class LGURLAgent(Agent):
    def __init__(self, unique_id, model, rl_config):
        super().__init__(unique_id, model)
        self.lr = rl_config['learning_rate']
        self.gamma = rl_config['discount_factor']
        self.epsilon = rl_config['epsilon']
        self.action_space = rl_config['action_space'] 
        self.q_table = defaultdict(lambda: np.zeros(len(self.action_space)))
        self.last_state = None
        self.last_action_idx = None
        
    def get_state(self, model):
        # SIMPLIFIED STATE: Global Average Compliance (0-20 scale)
        avg_compliance = np.mean([b.current_compliance_rate for b in model.barangays.values()])
        compliance_bin = int(avg_compliance * 20) # 20% -> 4
        
        budget_bin = int(model.annual_budget_remaining // 100000) 
        quarter = model.schedule.steps % STEPS_PER_EPISODE
        
        # Returns a tuple like (4, 15, 1) - Easy to learn!
        return (compliance_bin, budget_bin, quarter) 

    def choose_action(self, state):
        if random.random() < self.epsilon:
            action_idx = random.randrange(len(self.action_space))
        else:
            action_idx = np.argmax(self.q_table[state])
        self.last_state = state
        self.last_action_idx = action_idx
        return self.action_space[action_idx]

    def update_Q(self, new_state, reward):
        old_value = self.q_table[self.last_state][self.last_action_idx]
        next_max = np.max(self.q_table[new_state])
        new_value = old_value + self.lr * (reward + self.gamma * next_max - old_value)
        self.q_table[self.last_state][self.last_action_idx] = new_value

In [5]:
# --- CELL 5: SWM MODEL ---
class SWMModel(Model):
    def __init__(self, rl_config, fine_magnitude=500):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.running = True
        
        self.MAX_INCOME_PROXY = 50000 
        self.ENFORCEMENT_PERCEPTION_RATE = 0.5 
        self.IEC_EFFECTIVENESS = 0.05
        self.LGU_FINE_MAGNITUDE = fine_magnitude 

        self.current_id = 1000 
        self.RL_AGENT_ID = 999 
        
        self.rl_agent = LGURLAgent(self.RL_AGENT_ID, self, rl_config) 
        self.schedule.add(self.rl_agent)
        
        self.annual_budget_remaining = ANNUAL_SWM_BUDGET
        self.quarterly_budget = QUARTERLY_BUDGET
        
        self.barangays = {}
        self.household_agents = []
        self._initialize_agents()
        
        self.datacollector = DataCollector(
            model_reporters={
                "AvgCompliance": lambda m: np.mean([b.current_compliance_rate for b in m.barangays.values()]),
                "Total_Spent_This_Q": lambda m: m.last_cost,
                "Allocated_IEC": lambda m: m.last_alloc['IEC'],
                "Allocated_Enforce": lambda m: m.last_alloc['ENFORCE'],
                "Allocated_Incentive": lambda m: m.last_alloc['INCENTIVE'],
                "RLReward": lambda m: m.last_reward
            }
        )
        self.last_cost = 0
        self.last_reward = 0
        self.last_alloc = {'IEC': 0, 'ENFORCE': 0, 'INCENTIVE': 0}

    def _get_next_id(self):
        self.current_id += 1
        return self.current_id

    def _initialize_agents(self):
        temp_id = 0
        def get_next_temp_id():
            nonlocal temp_id
            temp_id += 1
            return temp_id
            
        for name, data in BARANGAY_DATA.items():
            num_h, local_budget, initial_compliance = data
            barangay_agent = BarangayAgent(get_next_temp_id(), self, name, [], local_budget, initial_compliance)
            self.schedule.add(barangay_agent)
            self.barangays[name] = barangay_agent
            
            for _ in range(num_h):
                h_agent = HouseholdAgent(self._get_next_id(), self, random.uniform(10000, self.MAX_INCOME_PROXY), 
                                         random.choice([0, 1, 2]), name)
                self.household_agents.append(h_agent)
                barangay_agent.households.append(h_agent)
                self.schedule.add(h_agent) 

    def step(self):
        # 1. RL Agent Decides
        current_state = self.rl_agent.get_state(self)
        policy_vector = self.rl_agent.choose_action(current_state) 
        
        total_spending = 0
        for i, b_name in enumerate(BARANGAY_DATA.keys()):
            b_agent = self.barangays[b_name]
            
            # Update Budgets
            b_agent.policy_allocation['IEC'] = policy_vector[i * 3 + 0]
            b_agent.policy_allocation['ENFORCE'] = policy_vector[i * 3 + 1]
            b_agent.policy_allocation['INCENTIVE'] = policy_vector[i * 3 + 2]
            total_spending += sum(b_agent.policy_allocation.values())

        # 2. Households React
        total_compliance = 0
        total_households = sum(len(b.households) for b in self.barangays.values())
        total_incentive_cost = 0
        
        for b_agent in self.barangays.values():
            
            # --- CONNECTION FIX: CALCULATE RISK ---
            # Budget -> Staff -> Probability
            enforcers = b_agent.policy_allocation['ENFORCE'] / COST_PER_ENFORCER_QUARTER
            risk_prob = min(1.0, 0.05 * enforcers) 
            
            for h_agent in b_agent.households:
                # PASS RISK PROBABILITY to Household
                h_agent.step(self.LGU_FINE_MAGNITUDE, PAYOUT_PER_COMPLIANT_HOUSEHOLD, risk_prob) 
            
            compliance, fined_count, incentive_cost, revenue = b_agent.aggregate_and_report()
            total_compliance += compliance * len(b_agent.households)
            total_incentive_cost += incentive_cost

            fined_rate = fined_count / len(b_agent.households) if len(b_agent.households) > 0 else 0
            for h_agent in b_agent.households:
                h_agent.update_tpb_constructs(b_agent.policy_allocation, fined_rate, compliance)

        # 3. Reward Calculation
        avg_compliance = total_compliance / total_households
        actual_total_cost = total_spending + total_incentive_cost 
        budget_deficit = max(0, actual_total_cost - self.quarterly_budget)
        
        reward = (ALPHA_COMPLIANCE * avg_compliance) - (BETA_COST * actual_total_cost) - (GAMMA_DEFICIT * budget_deficit)
        
        new_state = self.rl_agent.get_state(self)
        self.rl_agent.update_Q(new_state, reward)
        
        # 4. Updates
        self.schedule.steps += 1
        self.annual_budget_remaining -= actual_total_cost
        if self.schedule.steps % STEPS_PER_EPISODE == 0:
            self.annual_budget_remaining = ANNUAL_SWM_BUDGET
            
        self.last_cost = actual_total_cost
        self.last_reward = reward
        self.last_alloc = {
            'IEC': sum(b.policy_allocation['IEC'] for b in self.barangays.values()),
            'ENFORCE': sum(b.policy_allocation['ENFORCE'] for b in self.barangays.values()),
            'INCENTIVE': sum(b.policy_allocation['INCENTIVE'] for b in self.barangays.values())
        }
        self.datacollector.collect(self)

In [6]:
# Cell 6: Initialization and Iteration Control 💡

# --- SAFETY CHECK: Define Critical Global Constants ---
# Re-defining here ensures the functions below can access them reliably.
MAX_EPISODES = 5000 # Example value
QUARTERLY_BUDGET = 375000.0
NUM_BARANGAYS = 7
STEPS_PER_EPISODE = 4 

# --- RL Action Space Definition and Configuration ---

def generate_scenario_actions(iec_enabled, enforce_enabled, incentive_enabled):
    """Generates baseline and targeted (Heuristic) actions."""
    actions = []
    # Baseline Options (Equal Split)
    spending_levels = [0, 50000, 100000]
    for s_iec in spending_levels:
        for s_enforce in spending_levels:
            for s_incentive in spending_levels:
                total_alloc = s_iec + s_enforce + s_incentive
                if total_alloc <= QUARTERLY_BUDGET: 
                    action_vector = []
                    for _ in range(NUM_BARANGAYS):
                        action_vector.extend([s_iec/NUM_BARANGAYS, s_enforce/NUM_BARANGAYS, s_incentive/NUM_BARANGAYS])
                    actions.append(tuple(action_vector))

    # Heuristic Options (Targeted)
    if enforce_enabled: # Iron Fist
        iron_fist = []
        for _ in range(NUM_BARANGAYS): iron_fist.extend([0, 200000/NUM_BARANGAYS, 0])
        actions.append(tuple(iron_fist))
    if iec_enabled: # Educator
        educator = []
        for _ in range(NUM_BARANGAYS): educator.extend([200000/NUM_BARANGAYS, 0, 0])
        actions.append(tuple(educator))
    if incentive_enabled: # Populist
        populist = []
        for _ in range(NUM_BARANGAYS): populist.extend([0, 0, 200000/NUM_BARANGAYS])
        actions.append(tuple(populist))
    return actions

# Generate the actions first 
HYBRID_ACTIONS = generate_scenario_actions(True, True, True)

RL_CONFIG = {
    'learning_rate': 0.1, 
    'discount_factor': 0.9, 
    'epsilon': 1.0, 
    # CHANGE 1: Faster Decay (0.9995 -> 0.99)
    # This makes the agent "grow up" in 500 episodes instead of 5000.
    'epsilon_decay': 0.99, 
    'epsilon_min': 0.01, 
    'action_space': HYBRID_ACTIONS 
}

# --- INITIALIZE THE MODEL FOR ITERATIVE RUNNING ---

# Define and conditionally reset global model variables
global global_model
global training_results
global current_episode

if 'global_model' in globals():
    try:
        if global_model is not None:
            del global_model 
            print("Previous SWMModel instance deleted to clear scheduler.")
    except NameError:
        pass 

# Initialize global variables (Must be defined before Cell 7)
global_model = SWMModel(RL_CONFIG)
training_results = []
current_episode = 0

print(f"Model initialized. Target training episodes: {MAX_EPISODES}")
print(f"Total actions in the selected RL space: {len(HYBRID_ACTIONS)}")


# --- ITERATION CONTROL FUNCTION (Remains defined here) ---
def run_single_episode(model, episode_number):
    """Runs one full episode (4 quarters) of the ABM simulation and updates the RL Q-table."""
    
    if episode_number >= MAX_EPISODES:
        return None, True

    # Decay Epsilon 
    model.rl_agent.epsilon = max(
        model.rl_agent.epsilon * RL_CONFIG['epsilon_decay'],
        RL_CONFIG['epsilon_min']
    )
    
    # Run 4 Quarterly Steps (ABM Simulation)
    for _ in range(STEPS_PER_EPISODE):
        model.step() 

    # Record Results
    df = model.datacollector.get_model_vars_dataframe()
    episode_data = df.iloc[-1]
    
    return episode_data, False

# Output check: Status message for readiness
print("\nReady to begin full training loop in the next cell.")

Model initialized. Target training episodes: 5000
Total actions in the selected RL space: 30

Ready to begin full training loop in the next cell.


In [7]:
# Cell 7: Full Automatic Training Loop 🚀 (Runs 5000 Episodes)

# Import tqdm for a nice progress bar
from tqdm.notebook import tqdm 

# Check if the model is initialized (should be defined by Cell 6)
if 'global_model' not in globals() or global_model is None:
    print("ERROR: Model not initialized. Please run Cell 6 first.")
else:
    print(f"Starting training loop for {MAX_EPISODES} episodes...")

    # Start the loop from the current episode count up to MAX_EPISODES
    start_episode = current_episode
    
    # Use tqdm to create the progress bar
    for episode in tqdm(range(start_episode + 1, MAX_EPISODES + 1), initial=start_episode, total=MAX_EPISODES, desc="Training Progress"):
        
        # 1. Update the global counter
        current_episode = episode
        
        # 2. Run the episode
        new_data, is_finished = run_single_episode(global_model, current_episode)

        if is_finished:
            break
        
        # 3. Store results
        training_results.append(new_data)
        
        # 4. Optional: Print status periodically (e.g., every 100 episodes)
        if episode % 100 == 0:
            avg_compliance = new_data['AvgCompliance']
            epsilon = global_model.rl_agent.epsilon
            print(f"\n[Status] Episode {episode} | Avg Comp: {avg_compliance:.4f} | Epsilon: {epsilon:.4f}")


    print(f"\n--- FULL TRAINING COMPLETE ({current_episode} Episodes Run) ---")
    print("Proceed to Cell 8 for final analysis and plotting.")

Starting training loop for 5000 episodes...


Training Progress:   0%|          | 0/5000 [00:00<?, ?it/s]


[Status] Episode 100 | Avg Comp: 0.9551 | Epsilon: 0.3660

[Status] Episode 200 | Avg Comp: 0.9517 | Epsilon: 0.1340

[Status] Episode 300 | Avg Comp: 0.9591 | Epsilon: 0.0490

[Status] Episode 400 | Avg Comp: 0.9594 | Epsilon: 0.0180

[Status] Episode 500 | Avg Comp: 0.9600 | Epsilon: 0.0100

[Status] Episode 600 | Avg Comp: 0.9600 | Epsilon: 0.0100

[Status] Episode 700 | Avg Comp: 0.9600 | Epsilon: 0.0100

[Status] Episode 800 | Avg Comp: 0.9610 | Epsilon: 0.0100

[Status] Episode 900 | Avg Comp: 0.9612 | Epsilon: 0.0100

[Status] Episode 1000 | Avg Comp: 0.9614 | Epsilon: 0.0100

[Status] Episode 1100 | Avg Comp: 0.9614 | Epsilon: 0.0100

[Status] Episode 1200 | Avg Comp: 0.9614 | Epsilon: 0.0100

[Status] Episode 1300 | Avg Comp: 0.9622 | Epsilon: 0.0100

[Status] Episode 1400 | Avg Comp: 0.9629 | Epsilon: 0.0100

[Status] Episode 1500 | Avg Comp: 0.9629 | Epsilon: 0.0100

[Status] Episode 1600 | Avg Comp: 0.9631 | Epsilon: 0.0100

[Status] Episode 1700 | Avg Comp: 0.9639 | Epsil